In [ ]:
# Importar la librería pandas para el manejo de datos tabulares
import pandas as pd
# Importar la clase datetime y timedelta para el manejo de datos de tiempo
from datetime import datetime, timedelta

# Cargar el conjunto de datos desde el archivo CSV
url = "mantenimiento_modificado.csv"
df = pd.read_csv(url)

# Convertir las columnas de fecha a formato de fecha utilizando la función pd.to_datetime()
columnas = ["Fecha de Fallo 1", "Fecha de Fallo 2", "Fecha de Fallo 3", "Fecha de Fallo 4", "Fecha de Fallo 5"]
for col in columnas:
    df[col] = pd.to_datetime(df[col])

# Crear una nueva columna para la fecha del sexto fallo e inicializarla con valores NaT (Not a Time)
df["Fecha de Fallo 6"] = pd.NaT

# Iterar sobre cada fila del DataFrame
for _, row in df.iterrows():
    # Filtrar las fechas de fallo válidas para la máquina actual y convertirlas en una lista
    fechas_fallo = row[columnas].dropna().tolist()

    # Verificar si hay al menos cinco fechas de fallo disponibles para calcular el promedio
    if len(fechas_fallo) >= 5:
        # Calcular el promedio de días entre fallos
        promedio_dias_entre_fallos = sum((fechas_fallo[i] - fechas_fallo[i - 1]).days for i in range(1, len(fechas_fallo))) / (len(fechas_fallo) - 1)

        # Calcular la fecha estimada del sexto fallo sumando el promedio de días al último fallo registrado
        fecha_sexto_fallo = fechas_fallo[-1] + timedelta(days=promedio_dias_entre_fallos)

        # Asignar la fecha del sexto fallo calculada a la columna correspondiente en el DataFrame
        df.at[_, "Fecha de Fallo 6"] = fecha_sexto_fallo

# Mostrar el resultado final, mostrando únicamente las columnas 'ID Máquina' y 'Fecha de Fallo 6'
print(df[["ID Máquina", "Fecha de Fallo 6"]])


In [37]:
# Comparativa entre árbol de decisión y regresión lineal

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Cargar el conjunto de datos desde el archivo CSV
df = pd.read_csv("mantenimiento_modificado.csv")

# Eliminar columnas no numéricas
df = df.drop(columns=['ID Máquina', 'Tipo'])

# Convertir las fechas a tipo datetime
for col in ['Fecha de Fallo 1', 'Fecha de Fallo 2', 'Fecha de Fallo 3', 'Fecha de Fallo 4', 'Fecha de Fallo 5']:
    df[col] = pd.to_datetime(df[col])

# Extraer el año, el mes y el día de la semana de cada fecha
for col in ['Fecha de Fallo 1', 'Fecha de Fallo 2', 'Fecha de Fallo 3', 'Fecha de Fallo 4', 'Fecha de Fallo 5']:
    df[col + '_año'] = df[col].dt.year
    df[col + '_mes'] = df[col].dt.month
    df[col + '_día_semana'] = df[col].dt.weekday

# Eliminar las columnas originales de las fechas
df = df.drop(columns=['Fecha de Fallo 1', 'Fecha de Fallo 2', 'Fecha de Fallo 3', 'Fecha de Fallo 4', 'Fecha de Fallo 5'])

# Separar las características (X) y la variable objetivo (y)
X = df.drop(columns=['Fallo'])
y = df['Fallo']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir los modelos
modelo_lineal = LinearRegression()
modelo_arbol = DecisionTreeRegressor(random_state=42)

# Validación cruzada para el modelo de regresión lineal
lineales = cross_val_score(modelo_lineal, X_entrenamiento, y_entrenamiento, cv=7, scoring='neg_mean_squared_error')
mse_promedio_lineal = -lineales.mean()

# Validación cruzada para el modelo de árbol de decisión
arbol = cross_val_score(modelo_arbol, X_entrenamiento, y_entrenamiento, cv=7, scoring='neg_mean_squared_error')
mse_promedio_arbol = -arbol.mean()

# Entrenar los modelos con los datos de entrenamiento
modelo_lineal.fit(X_entrenamiento, y_entrenamiento)
modelo_arbol.fit(X_entrenamiento, y_entrenamiento)

# Predecir con los datos de prueba y calcular el MSE
prediccion_lineal = modelo_lineal.predict(X_prueba)
mse_prueba_lineal = mean_squared_error(y_prueba, prediccion_lineal)

prediccion_arbol = modelo_arbol.predict(X_prueba)
mse_prueba_arbol = mean_squared_error(y_prueba, prediccion_arbol)

# Imprimir los resultados
print("Error cuadratico medio:")
print("MSE promedio en validación cruzada (conjunto de entrenamiento):")
print("Regresión Lineal:", mse_promedio_lineal)
print("Árbol de Decisión:", mse_promedio_arbol)
print("\nMSE en conjunto de prueba:")
print("Regresión Lineal:", mse_prueba_lineal)
print("Árbol de Decisión:", mse_prueba_arbol)


Error cuadratico medio:
MSE promedio en validación cruzada (conjunto de entrenamiento):
Regresión Lineal: 1.521384708446059
Árbol de Decisión: 0.4571428571428572

MSE en conjunto de prueba:
Regresión Lineal: 1.4664058628232453
Árbol de Decisión: 1.0


In [39]:
# Comparativa entre Árbol de Decisión y SVR

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

# Cargar los datos
df = pd.read_csv("mantenimiento_modificado.csv")

# Eliminar la columna "Fecha de Fallo 1"
df = df.drop(columns=["Fecha de Fallo 1"])

# Convertir las fechas a números
for col in df.columns:
    if 'Fecha' in col:
        df[col] = pd.to_datetime(df[col]).astype(int) / 10**9  # Convertir a segundos desde la época

# Dividir los datos en características (X) y objetivo (y)
X = df.drop(columns=["ID Máquina", "Fallo"])
y = df["Fallo"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos de Árbol de Decisiones y SVR
modelo_arbol = DecisionTreeRegressor()
modelo_svm = SVR()

# Entrenar los modelos
modelo_arbol.fit(X_entrenamiento, y_entrenamiento)
modelo_svm.fit(X_entrenamiento, y_entrenamiento)

# Validación cruzada para los modelos
arbol = cross_val_score(modelo_arbol, X_entrenamiento, y_entrenamiento, cv=7, scoring='neg_mean_squared_error')
svm = cross_val_score(modelo_svm, X_entrenamiento, y_entrenamiento, cv=7, scoring='neg_mean_squared_error')

# Calcular el MSE promedio en validación cruzada
mse_promedio_arbol = -arbol.mean()
mse_promedio_svm = -svm.mean()

print("Error cuadratico medio:")
print("MSE promedio en validación cruzada (conjunto de entrenamiento):")
print(f"Árbol de Decisión: {mse_promedio_arbol}")
print(f"SVR: {mse_promedio_svm}")

# Predecir con los datos de prueba y calcular el MSE
prediccion_arbol = modelo_arbol.predict(X_prueba)
prediccion_svm = modelo_svm.predict(X_prueba)

mse_prueba_arbol = mean_squared_error(y_prueba, prediccion_arbol)
mse_prueba_svm = mean_squared_error(y_prueba, prediccion_svm)


print("\nMSE en conjunto de prueba:")
print(f"Árbol de Decisión: {mse_prueba_arbol}")
print(f"SVR: {mse_prueba_svm}")


Error cuadratico medio:
MSE promedio en validación cruzada (conjunto de entrenamiento):
Árbol de Decisión: 0.5642857142857143
SVR: 0.35000155740361966

MSE en conjunto de prueba:
Árbol de Decisión: 2.625
SVR: 1.0100263550357762


In [ ]:
# Prueba con SARIMAX

import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Cargar los datos desde el archivo CSV
df = pd.read_csv("mantenimiento_modificado.csv")

# Convertir las columnas de fecha a objetos datetime y establecer la frecuencia
for col in df.columns[3:]:
    df[col] = pd.to_datetime(df[col])

# Seleccionar la columna de fechas de fallo para el análisis
columna_fecha = 'Fecha de Fallo 5'
serie_temporal = df[columna_fecha]

# Ajustar el modelo SARIMA con parámetros iniciales modificados
try:
    modelo = SARIMAX(serie_temporal, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    resultado = modelo.fit(maxiter=200)  # Aumentar el número máximo de iteraciones
except Exception as e:
    print("Error al ajustar el modelo SARIMA:", e)
    resultado = None

# Hacer predicciones
if resultado:
    # Generar fechas de predicción en el formato adecuado
    fechas_prediccion = pd.date_range(start=serie_temporal.index[0], periods=len(serie_temporal) + 5, freq='D')

    try:
        predicciones = resultado.predict(start=0, end=len(serie_temporal) + 5)
        predicciones.index = fechas_prediccion
        print(predicciones)
    except Exception as e:
        print("Error al realizar las predicciones:", e)
else:
    print("¡Advertencia! No hay suficientes datos para ajustar el modelo SARIMA.")


Error al ajustar el modelo SARIMA: ufunc 'subtract' cannot use operands with types dtype('<m8[ns]') and dtype('O')
¡Advertencia! No hay suficientes datos para ajustar el modelo SARIMA.


In [32]:
# Regresión Lineal

# Importar la librería pandas para el manejo de datos tabulares
import pandas as pd
# Importar la clase LinearRegression del módulo sklearn.linear_model para realizar regresión lineal
from sklearn.linear_model import LinearRegression
from tabulate import tabulate

# Cargar el conjunto de datos desde el archivo CSV
url = "mantenimiento_modificado.csv"
df = pd.read_csv(url)

# Convertir las columnas de fecha a formato de fecha utilizando la función pd.to_datetime()
columnas = ["Fecha de Fallo 1", "Fecha de Fallo 2", "Fecha de Fallo 3", "Fecha de Fallo 4", "Fecha de Fallo 5"]
for col in columnas:
    df[col] = pd.to_datetime(df[col])

# Crear una nueva columna para la fecha del sexto fallo e inicializarla con valores NaT (Not a Time)
df["Fecha de Fallo 6"] = pd.NaT

# Iterar sobre cada fila del DataFrame
for index, fila in df.iterrows():
    # Filtrar las fechas de fallo válidas para la fila actual y convertirlas en una lista
    fechas_fallo = fila[columnas].dropna().tolist()

    # Verificar si hay al menos cinco fechas de fallo disponibles para calcular la regresión lineal
    if len(fechas_fallo) >= 5:
        # Calcular las diferencias de tiempo en días entre fallos consecutivos
        diferencias_dias = [(fechas_fallo[i] - fechas_fallo[i - 1]).days for i in range(1, len(fechas_fallo))]

        # Crear las características (X) y el objetivo (y) para el modelo de regresión
        X = [[i] for i in range(1, len(fechas_fallo))]  # Números secuenciales de fallos
        y = diferencias_dias

        # Entrenar el modelo de regresión lineal
        reg = LinearRegression().fit(X, y)

        # Predecir los días del sexto fallo utilizando el modelo entrenado
        fecha_fallo_prediccion = fechas_fallo[-1] + pd.Timedelta(days=int(reg.predict([[6]])[0]))

        # Limitar la predicción a 90 días después del quinto fallo
        fecha_quinto_fallo = fechas_fallo[-1]
        fecha_limite = fecha_quinto_fallo + pd.Timedelta(days=90)
        fecha_fallo = min(fecha_fallo_prediccion, fecha_limite)

        # Asignar la fecha de fallo al DataFrame
        df.at[index, "Fecha de Fallo 6"] = fecha_fallo

# Ordenar el DataFrame por Tipo y Fallo
df.sort_values(by=["Tipo", "Fallo"], inplace=True)

# Formatear las fechas sin las horas en el DataFrame original y en la tabla adicional
for col in columnas + ["Fecha de Fallo 6"]:
    df[col] = df[col].dt.strftime('%Y-%m-%d')

# Crear una tabla adicional con las columnas "maquina", "tipo", "fallo" y "proximo fallo"
tabla_proximos_fallos = df[["ID Máquina", "Tipo", "Fallo", "Fecha de Fallo 6"]].rename(columns={"Fecha de Fallo 6": "Próximo Fallo"})

# Mostrar el resultado final como una tabla ordenada por Tipo y Fallo de forma ascendente
print("Ordenado por Tipo y Fallo (Ascendente):\n")
print(tabulate(df, headers='keys', tablefmt='pretty'))

# Mostrar la tabla adicional con un formato más bonito
print("\nTabla de Próximos Fallos:\n")
print(tabulate(tabla_proximos_fallos, headers='keys', tablefmt='pretty'))


Ordenado por Tipo y Fallo (Ascendente):

+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|    | ID Máquina | Tipo | Fallo | Fecha de Fallo 1 | Fecha de Fallo 2 | Fecha de Fallo 3 | Fecha de Fallo 4 | Fecha de Fallo 5 | Fecha de Fallo 6 |
+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
| 0  |   L47762   |  1   |   1   |    2022-10-19    |    2022-11-12    |    2022-12-06    |    2022-12-30    |    2023-01-23    |    2023-02-16    |
| 1  |   L50530   |  1   |   1   |    2022-12-02    |    2022-12-26    |    2023-01-19    |    2023-02-12    |    2023-03-08    |    2023-04-01    |
| 3  |   L51911   |  1   |   1   |    2023-03-02    |    2023-03-26    |    2023-04-19    |    2023-05-13    |    2023-06-06    |    2023-06-30    |
| 4  |   L56594   |  1   |   1   |    2023-02-21    |    2023-03-

In [31]:
# Árbol de Decisiones

# Importar la librería pandas para el manejo de datos tabulares
import pandas as pd
# Importar la clase DecisionTreeRegressor del módulo sklearn.tree para realizar la regresión con árboles de decisión
from sklearn.tree import DecisionTreeRegressor
from tabulate import tabulate

# Cargar el conjunto de datos desde el archivo CSV
url = "mantenimiento_modificado.csv"
df = pd.read_csv(url)

# Convertir las columnas de fecha a formato de fecha utilizando la función pd.to_datetime()
columnas = ["Fecha de Fallo 1", "Fecha de Fallo 2", "Fecha de Fallo 3", "Fecha de Fallo 4", "Fecha de Fallo 5"]
for col in columnas:
    df[col] = pd.to_datetime(df[col])

# Crear una nueva columna para la fecha del sexto fallo e inicializarla con valores NaT (Not a Time)
df["Fecha de Fallo 6"] = pd.NaT

# Iterar sobre cada fila del DataFrame
for index, fila in df.iterrows():
    # Filtrar las fechas de fallo válidas para la fila actual y convertirlas en una lista
    fechas_fallo = fila[columnas].dropna().tolist()

    # Verificar si hay al menos cinco fechas de fallo disponibles para calcular la regresión lineal
    if len(fechas_fallo) >= 5:
        # Calcular las diferencias de tiempo en días entre fallos consecutivos
        diferencias_dias = [(fechas_fallo[i] - fechas_fallo[i - 1]).days for i in range(1, len(fechas_fallo))]

        # Crear las características (X) y el objetivo (y) para el modelo de regresión
        X = [[i] for i in range(1, len(fechas_fallo))]  # Números secuenciales de fallos
        y = diferencias_dias

        # Entrenar el modelo de regresión con Árbol de Decisiones
        tree_reg = DecisionTreeRegressor().fit(X, y)

        # Predecir los días del sexto fallo utilizando el modelo entrenado
        fecha_fallo_prediccion = fechas_fallo[-1] + pd.Timedelta(days=int(tree_reg.predict([[6]])[0]))

        # Limitar la predicción a 90 días después del quinto fallo
        fecha_quinto_fallo = fechas_fallo[-1]
        fecha_limite = fecha_quinto_fallo + pd.Timedelta(days=90)
        fecha_fallo = min(fecha_fallo_prediccion, fecha_limite)

        # Asignar la fecha de fallo al DataFrame
        df.at[index, "Fecha de Fallo 6"] = fecha_fallo

# Ordenar el DataFrame por Tipo y Fallo
df.sort_values(by=["Tipo", "Fallo"], inplace=True)

# Formatear las fechas sin las horas en el DataFrame original
for col in columnas + ["Fecha de Fallo 6"]:
    df[col] = df[col].dt.strftime('%Y-%m-%d')

# Mostrar el resultado final como una tabla ordenada por Tipo y Fallo de forma ascendente
print("Ordenado por Tipo y Fallo (Ascendente):\n")
print(tabulate(df, headers='keys', tablefmt='pretty'))

# Crear un nuevo DataFrame para almacenar las fechas del sexto fallo junto con la información de "maquina", "tipo" y "fallo"
nuevo_df = pd.DataFrame(columns=["maquina", "tipo", "fallo", "proximo error"])

# Iterar sobre cada fila del DataFrame original
frames = []  # Lista para almacenar los DataFrames individuales de cada fila
for index, fila in df.iterrows():
    # Obtener la información relevante de la fila actual
    maquina = fila["ID Máquina"]
    tipo = fila["Tipo"]
    fallo = fila["Fallo"]
    proximo_error = fila["Fecha de Fallo 6"]  # Obtenemos la fecha del sexto fallo del DataFrame original

    # Agregar una nueva fila al nuevo DataFrame con la información obtenida
    frames.append(pd.DataFrame({"maquina": [maquina], "tipo": [tipo], "fallo": [fallo], "proximo error": [proximo_error]}))

# Concatenar todos los DataFrames individuales en uno solo
nuevo_df = pd.concat(frames, ignore_index=True)

# Mostrar el nuevo DataFrame con la información del sexto fallo
print("\nTabla adicional con las fechas del sexto fallo:\n")
print(tabulate(nuevo_df, headers='keys', tablefmt='pretty'))


Ordenado por Tipo y Fallo (Ascendente):

+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|    | ID Máquina | Tipo | Fallo | Fecha de Fallo 1 | Fecha de Fallo 2 | Fecha de Fallo 3 | Fecha de Fallo 4 | Fecha de Fallo 5 | Fecha de Fallo 6 |
+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
| 0  |   L47762   |  1   |   1   |    2022-10-19    |    2022-11-12    |    2022-12-06    |    2022-12-30    |    2023-01-23    |    2023-02-16    |
| 1  |   L50530   |  1   |   1   |    2022-12-02    |    2022-12-26    |    2023-01-19    |    2023-02-12    |    2023-03-08    |    2023-04-01    |
| 3  |   L51911   |  1   |   1   |    2023-03-02    |    2023-03-26    |    2023-04-19    |    2023-05-13    |    2023-06-06    |    2023-06-30    |
| 4  |   L56594   |  1   |   1   |    2023-02-21    |    2023-03-

In [30]:
# SVR (Support Vector Regressor)

# Importar la librería pandas para el manejo de datos tabulares
import pandas as pd
# Importar la clase SVR del módulo sklearn.svm para realizar la regresión con Support Vector Regressor
from sklearn.svm import SVR
from tabulate import tabulate

# Cargar el conjunto de datos desde el archivo CSV
url = "mantenimiento_modificado.csv"
df = pd.read_csv(url)

# Convertir las columnas de fecha a formato de fecha utilizando la función pd.to_datetime()
columnas = ["Fecha de Fallo 1", "Fecha de Fallo 2", "Fecha de Fallo 3", "Fecha de Fallo 4", "Fecha de Fallo 5"]
for col in columnas:
    df[col] = pd.to_datetime(df[col])

# Crear una nueva columna para la fecha del sexto fallo e inicializarla con valores NaT (Not a Time)
df["Fecha de Fallo 6"] = pd.NaT

# Iterar sobre cada fila del DataFrame
for index, fila in df.iterrows():
    # Filtrar las fechas de fallo válidas para la fila actual y convertirlas en una lista
    fechas_fallo = fila[columnas].dropna().tolist()

    # Verificar si hay al menos cinco fechas de fallo disponibles para calcular la regresión lineal
    if len(fechas_fallo) >= 5:
        # Calcular las diferencias de tiempo en días entre fallos consecutivos
        diferencias_dias = [(fechas_fallo[i] - fechas_fallo[i - 1]).days for i in range(1, len(fechas_fallo))]

        # Crear las características (X) y el objetivo (y) para el modelo de regresión
        X = [[i] for i in range(1, len(fechas_fallo))]  # Números secuenciales de fallos
        y = diferencias_dias

        # Entrenar el modelo de regresión con SVR (Support Vector Regressor)
        svr_reg = SVR().fit(X, y)

        # Predecir los días del sexto fallo utilizando el modelo entrenado
        fecha_fallo_prediccion = fechas_fallo[-1] + pd.Timedelta(days=int(svr_reg.predict([[6]])[0]))

        # Limitar la predicción a 90 días después del quinto fallo
        fecha_quinto_fallo = fechas_fallo[-1]
        fecha_limite = fecha_quinto_fallo + pd.Timedelta(days=90)
        fecha_fallo = min(fecha_fallo_prediccion, fecha_limite)

        # Asignar la fecha de fallo al DataFrame
        df.at[index, "Fecha de Fallo 6"] = fecha_fallo

# Ordenar el DataFrame por Tipo y Fallo
df.sort_values(by=["Tipo", "Fallo"], inplace=True)

# Formatear las fechas sin las horas en el DataFrame original
for col in columnas + ["Fecha de Fallo 6"]:
    df[col] = df[col].dt.strftime('%Y-%m-%d')

# Mostrar el resultado final como una tabla ordenada por Tipo y Fallo de forma ascendente
print("Ordenado por Tipo y Fallo (Ascendente):\n")
print(tabulate(df, headers='keys', tablefmt='pretty'))

# Crear un nuevo DataFrame para almacenar las fechas del sexto fallo junto con la información de "maquina", "tipo" y "fallo"
nuevo_df = pd.DataFrame(columns=["maquina", "tipo", "fallo", "proximo error"])

# Iterar sobre cada fila del DataFrame original
frames = []  # Lista para almacenar los DataFrames individuales de cada fila
for index, fila in df.iterrows():
    # Obtener la información relevante de la fila actual
    maquina = fila["ID Máquina"]
    tipo = fila["Tipo"]
    fallo = fila["Fallo"]
    proximo_error = fila["Fecha de Fallo 6"]  # Obtenemos la fecha del sexto fallo del DataFrame original

    # Agregar una nueva fila al nuevo DataFrame con la información obtenida
    frames.append(pd.DataFrame({"maquina": [maquina], "tipo": [tipo], "fallo": [fallo], "proximo error": [proximo_error]}))

# Concatenar todos los DataFrames individuales en uno solo
nuevo_df = pd.concat(frames, ignore_index=True)

# Mostrar el nuevo DataFrame con la información del sexto fallo
print("\nTabla adicional con las fechas del sexto fallo:\n")
print(tabulate(nuevo_df, headers='keys', tablefmt='pretty'))


Ordenado por Tipo y Fallo (Ascendente):

+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
|    | ID Máquina | Tipo | Fallo | Fecha de Fallo 1 | Fecha de Fallo 2 | Fecha de Fallo 3 | Fecha de Fallo 4 | Fecha de Fallo 5 | Fecha de Fallo 6 |
+----+------------+------+-------+------------------+------------------+------------------+------------------+------------------+------------------+
| 0  |   L47762   |  1   |   1   |    2022-10-19    |    2022-11-12    |    2022-12-06    |    2022-12-30    |    2023-01-23    |    2023-02-16    |
| 1  |   L50530   |  1   |   1   |    2022-12-02    |    2022-12-26    |    2023-01-19    |    2023-02-12    |    2023-03-08    |    2023-04-01    |
| 3  |   L51911   |  1   |   1   |    2023-03-02    |    2023-03-26    |    2023-04-19    |    2023-05-13    |    2023-06-06    |    2023-06-30    |
| 4  |   L56594   |  1   |   1   |    2023-02-21    |    2023-03-